In [10]:
import tkinter as tk
from tkinter import messagebox
from datetime import date
import time

def clear_selection():
    for var in group_vars.values():
            var.set(0)

def select_all_groups():
    if select_all_var.get() == 1:
        for var in group_vars.values():
            var.set(1)
        for group_key, var in group_vars.items():
            if var.get() == 1:
                add_task()  # 체크된 그룹에 대해 할 일 추가
    else:
        for var in group_vars.values():
            var.set(0)

def on_listbox_select(event):
    selected_listbox = event.widget
    selected_indices = selected_listbox.curselection()

    for listbox in listboxes.values():
        listbox.selection_clear(0, tk.END)  # Clear previous selections

    for index in selected_indices:
        selected_listbox.selection_set(index)  # Select clicked item

    update_delete_button_state()  # Update the state of the delete button

def load_tasks():
    for group_key, listbox in listboxes.items():
        listbox.delete(0, tk.END)  # Delete existing tasks in the listbox
        try:
            with open(f"todo/{group_key}.txt", "r", encoding="utf-8") as file:  # Specify the correct file path
                tasks = file.readlines()
                for task in tasks:
                    listbox.insert(tk.END, task.strip())  # Add each task to the listbox
        except FileNotFoundError:
            pass


def get_selected_group2():
    selected_groups = [group_key for group_key, var in group_vars.items() if var.get()]
    if selected_groups:
        return selected_groups[0]  # Return the first selected group
    else:
        return None  # Return None if no group is selected
    

def get_selected_group():
    selected_groups = [group_key for group_key, listbox in listboxes.items() if listbox.curselection()]
    if selected_groups:
        return selected_groups[0]  # Return the first selected group
    else:
        return None  # Return None if no group is selected

def delete_task():
    selected_group_key = get_selected_group()

    if selected_group_key:
        selected_listbox = listboxes[selected_group_key]
        selected_indices = selected_listbox.curselection()

        if selected_indices:
            for index in selected_indices[::-1]:  # Reverse the indices to delete from the end first
                selected_listbox.delete(index)
            dialog = tk.Toplevel(root)
            dialog.title("Custom Dialog")
            
            label = tk.Label(dialog, text="삭제 되었습니다..")
            label.pack(padx=200, pady=200)
            
            root.after(1000, lambda: close_dialog(dialog))
        else:
            messagebox.showwarning("Warning", "작업 목록을 선택하시오.")
    else:
        messagebox.showwarning("Warning", "작업 목록을 선택하시오")

    update_delete_button_state()  # Update the state of the delete button

def add_task():
    task = entry.get()
    selected_groups = [group_key for group_key, var in group_vars.items() if var.get()]

    if task and selected_groups:
        for group_key in selected_groups:
            group_title = group_dict[group_key]
            task_with_group = f"{task}"
            current_listbox = listboxes[group_key]
            current_listbox.insert(tk.END, task_with_group)
        entry.delete(0, tk.END)
        
def save_tasks():
    for group_key, listbox in listboxes.items():
        tasks = listbox.get(0, tk.END)
        filename = f"todo/{group_key}.txt"

        with open(filename, "w", encoding="utf-8") as file:
            for task in tasks:
                file.write(f"{task}\n")
                dialog = tk.Toplevel(root)
                dialog.title("Custom Dialog")
                
                label = tk.Label(dialog, text="저장 되었습니다..")
                label.pack(padx=200, pady=200)
                
                root.after(1000, lambda: close_dialog(dialog))
    

def update_delete_button_state():
    selected_group_key = get_selected_group()

    if selected_group_key:
        selected_listbox = listboxes[selected_group_key]
        selected_indices = selected_listbox.curselection()

        if selected_indices:
            delete_button.config(state=tk.NORMAL)  # Enable the delete button
        else:
            delete_button.config(state=tk.DISABLED)  # Disable the delete button
    else:
        delete_button.config(state=tk.DISABLED)  # Disable the delete button

def on_closing():
    if messagebox.askyesno("닫기", "닫기 전에 저장 하시겠습니까?"):
        save_tasks()
        root.destroy()
    else:
        root.destroy()



root = tk.Tk()
root.title("Todo List")
# Create the main window
root.protocol("WM_DELETE_WINDOW", on_closing)

# Group information
group_dict = {
    "ISPNF01": "ISPNF01",
    "ISPNF02": "ISPNF02",
    "ISPNF03": "ISPNF03",
    "ISPNF04": "ISPNF04",
    "ISPNF05": "ISPNF05",
    "ISPNF06": "ISPNF06",
    "ISDMF01": "ISDMF01",
    "ISDMF02": "ISDMF02",
    "ISPNV01": "ISPNV01",
    "ISPNV02": "ISPNV02",
    "ISISF01": "ISISF01",
    "ISISF02": "ISISF02",
    "ISISF03": "ISISF03",
    "ISHHF01": "ISHHF01",
    "ISHHF02": "ISHHF02",
    "ISISV01": "ISISV01",
    "ISISV02": "ISISV02",
    "ISRAF01": "ISRAF01",
    "ISSAF01": "ISSAF01",
    "ISSAF02": "ISSAF02",
    "ISRAV01": "ISRAV01",
    "ISRSV01": "ISRSV01",
    "ISDAV01": "ISDAV01"
}


# 그룹 선택 체크 박스
group_frame = tk.Frame(root)
group_frame.pack(pady=1)

# 그룹 체크 박스 배치
select_all_var = tk.IntVar()
select_all_check = tk.Checkbutton(group_frame, text="전체 선택", variable=select_all_var, onvalue=1, offvalue=0,
                                  command=select_all_groups)
select_all_check.grid(row=0, column=0, padx=1)

group_vars = {}
row_count = 1  # row_count를 1로 초기화
column_count = 0
for group_key, group_title in group_dict.items():
    var = tk.IntVar()
    group_check = tk.Checkbutton(group_frame, text=group_title, variable=var, onvalue=1, offvalue=0, width=10)
    group_check.grid(row=row_count, column=column_count, padx=5, pady=5)
    group_vars[group_key] = var

    column_count += 1
    if column_count >= 12:
        row_count += 1
        column_count = 0

clear_selection_button = tk.Button(group_frame, text="체크 해제", command=clear_selection)
clear_selection_button.grid(row=0, column=1, padx=5)


## 커스텀 버튼 시작 %%
def CEN_DPN():
    for group_key, var in group_vars.items():
        if group_key in ["ISPNF01","ISPNF02","ISPNF03","ISPNF04","ISPNF05","ISPNF06","ISDMF01","ISDMF02","ISPNV01","ISPNV02"]:
            if var.get() == 0:
                var.set(1)
            else:
                 var.set(0)
# 선택 버튼
select_specific_button = tk.Button(group_frame, text="DPN", command=CEN_DPN)
select_specific_button.grid(row=0, column=2)

def CEN_ISSG():
    for group_key, var in group_vars.items():
        if group_key in ["ISISF01","ISISF02","ISISF03","ISHHF01","ISHHF02","ISISV01","ISISV02"]:
            if var.get() == 0:
                var.set(1)
            else:
                 var.set(0)
# 선택 버튼
select_specific_button = tk.Button(group_frame, text="ISSG", command=CEN_ISSG)
select_specific_button.grid(row=0, column=3)

def Van():
    for group_key, var in group_vars.items():
        if group_key in ["ISRAF01", "ISSAF01", "ISSAF02", "ISRAV01", "ISRSV01", "ISDAV01"]:
            if var.get() == 0:
                var.set(1)
            else:
                 var.set(0)
# 선택 버튼
select_specific_button = tk.Button(group_frame, text="Vantage", command=Van)
select_specific_button.grid(row=0, column=4)
## 커스텀 버튼 종료 %%

# Listbox frame
listbox_frame = tk.Frame(root)
listbox_frame.pack(pady=10)

# Listboxes
listboxes = {}
row_count = 0
column_count = 0
for group_key, group_title in group_dict.items():
    listbox = tk.Listbox(listbox_frame, width=40, height=10, selectbackground="light blue", selectmode=tk.MULTIPLE)
    listbox.grid(row=row_count, column=column_count, padx=5, pady=5)
    title_label = tk.Label(listbox_frame, text=group_title)
    title_label.grid(row=row_count, column=column_count, padx=5, pady=5)
    listbox.bind("<<ListboxSelect>>", on_listbox_select)
    listboxes[group_key] = listbox

    column_count += 1
    if column_count >= 6:
        row_count += 1
        column_count = 0

# Scrollbar
scrollbar = tk.Scrollbar(listbox_frame)
scrollbar.grid(row=row_count, column=column_count, sticky="NS")

for listbox in listboxes.values():
    listbox.config(yscrollcommand=scrollbar.set)
    scrollbar.config(command=listbox.yview)

def close_dialog(dialog):
    dialog.destroy()

def comp():
    selected_group_key = get_selected_group()

    if selected_group_key:
        selected_listbox = listboxes[selected_group_key]
        selected_indices = selected_listbox.curselection()

        if selected_indices:
            selected_tasks = [selected_listbox.get(index) for index in selected_indices]
            today = date.today().strftime("%Y-%m-%d")

            with open("todo/comp.txt", "a", encoding="utf-8") as file:
                for task in selected_tasks:
                    file.write(f"{today} - {selected_group_key} - {task}\n")
            
            dialog = tk.Toplevel(root)
            dialog.title("Custom Dialog")
            
            label = tk.Label(dialog, text="완료 되었습니다..")
            label.pack(padx=200, pady=200)
            
            root.after(1000, lambda: close_dialog(dialog))
            
            for index in selected_indices[::-1]:
                selected_listbox.delete(index)  # Delete the completed tasks

    else:
        messagebox.showwarning("Warning", "작업 목록을 선택하시오.")

    update_delete_button_state()  # Update the state of the delete button


# Task entry
entry_frame = tk.Frame(root)
entry_frame.pack(pady=10)
label = tk.Label(entry_frame, text="할일 입력 창", padx=10)
label.pack(side="left")
entry = tk.Entry(entry_frame, font=("Helvetica", 12))
entry.pack(side="left", pady=5, padx=10)
add_button = tk.Button(entry_frame, text="추가", command=add_task)
add_button.pack(side="left" , pady=5, padx=10)

# Buttons
button_frame = tk.Frame(root)
button_frame.pack(pady=10)

delete_button = tk.Button(button_frame, text="삭제", command=delete_task, state=tk.DISABLED)
delete_button.grid(row=0, column=1, padx=20)

# save_button = tk.Button(button_frame, text="저장", command=save_tasks)
# save_button.grid(row=0, column=2, padx=20)

comp_button = tk.Button(button_frame, text="완료", command=comp, state=tk.NORMAL)
comp_button.grid(row=0, column=5, padx=20)



# Load tasks
load_tasks()

root.mainloop()
